In [1]:
import pandas as pd
from tabulate import tabulate
import os
import io
import subprocess

In [77]:
lista_codigos_cargos = ['CCD',
                        'CCS',
                        'CYC',
                        'CYT',
                        'EAD',
                        'ECS',
                        'IAC',
                        'OCL',
                        'OCS',
                        'ODM',
                        'TCI',
                        '/PR',
                        '+3E',
                        'CCC',
                        'PEE',
                        'PRA',
                        'PRT',
                        'YCS',
                        '/3D',
                        '/3E',
                        'CMN',
                        'EDP',
                        'MKS',
                        'TLC']
lista_codigos_cargos


['CCD',
 'CCS',
 'CYC',
 'CYT',
 'EAD',
 'ECS',
 'IAC',
 'OCL',
 'OCS',
 'ODM',
 'TCI',
 '/PR',
 '+3E',
 'CCC',
 'PEE',
 'PRA',
 'PRT',
 'YCS',
 '/3D',
 '/3E',
 'CMN',
 'EDP',
 'MKS',
 'TLC']

In [59]:
# lista_codigos_cargos_formateados = "*%20*".join(lista_codigos_cargos)
# lista_codigos_cargos_formateados
# lista_nombres_cargos = ['precep']
# lista_nombres_cargos_formateados = "%20".join(lista_nombres_cargos)
# lista_nombres_cargos_formateados

In [60]:
lista_distritos = ['berazategui','quilmes','avellaneda']
lista_distritos_formateados = "%20".join(lista_distritos)
lista_distritos_formateados

'berazategui%20quilmes%20avellaneda'

In [25]:
result = subprocess.run(f'''curl 'https://servicios3.abc.gob.ar/valoracion.docente/api/apd.oferta.encabezado/select?q=*%3A*&rows=10000&facet=true&facet.field=descdistrito&facet.field=descnivelmodalidad&facet.field=cargo&facet.field=estado&facet.limit=20000&facet.mincount=1&json.nl=map&fq=descdistrito%3A{lista_distritos_formateados}&fq=estado:Publicada&wt=csv' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Accept-Language: es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3' --compressed -H 'Connection: keep-alive' -H 'Cookie: _ga=GA1.3.1448501204.1598045085; _gid=GA1.3.1833139648.1598875839; IPCZQX034ee7c6fd=0100c3000a1432ac8ea31b21c67f7acb521691f8; ZNPCQ003-33383400=a89f2a51; ZNPCQ003-32383300=4e5b6dbe' -H 'Upgrade-Insecure-Requests: 1' | iconv -f iso-8859-1 -t UTF-8//TRANSLIT''', shell=True, capture_output=True, text=True)


In [78]:
df_sin_filtrar = pd.read_csv(io.StringIO(result.stdout))

In [79]:
df_sin_filtrar["areaincumbencia"] = df_sin_filtrar["areaincumbencia"].str.replace(" ","") #Por las dudas
df_sin_filtrar["areaincumbencia"] = df_sin_filtrar["areaincumbencia"].str.upper() #Por las dudas

In [80]:
df = df_sin_filtrar[df_sin_filtrar["areaincumbencia"].isin(lista_codigos_cargos)]

In [81]:
df

,estado,tipooferta,jornada,ige,miercoles,martes,acargodireccion,cuilautor,supl_hasta,turno,...,reemp_motivo,descdistrito,jueves,nivelmodalidad,viernes,descripcionarea,vuelta,descripcioncargo,ult_movimiento,_version_
3,Publicada,DESIGNACIONES DOCENTES,JS,3116245,9.30 A 10.30,NaN,No,20147633158,9999-12-30T12:30:21.887Z,M,...,NaN,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:30:23.140Z,1826390705143021572
4,Publicada,DESIGNACIONES DOCENTES,JS,3116363,NaN,NaN,No,20275425894,2026-02-28T00:00:00Z,M,...,Licencia no medica por ARTICULO 115(ART115),BERAZATEGUI,7:40 A 9:40,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:39:28.980Z,1826390705143021573
16,Publicada,DESIGNACIONES DOCENTES,JS,3116556,NaN,11:40-12:40,No,20147633158,2026-02-28T00:00:00Z,M,...,Relevo por servicios provisorios orden tecnico...,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:56:30.890Z,1826390706040602628
22,Publicada,DESIGNACIONES DOCENTES,JS,3117114,NaN,21:25 A 22:15,No,20275425894,9999-12-30T13:43:48.367Z,V,...,NaN,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T10:43:46.290Z,1826390706051088384
25,Publicada,DESIGNACIONES DOCENTES,JS,3116770,NaN,NaN,No,20147633158,2026-02-28T00:00:00Z,T,...,Licencia no medica por ARTICULO 114(ART114NOme...,BERAZATEGUI,NaN,E,17:20 A 18:20,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T10:13:01.070Z,1826390706449547264
26,Publicada,DESIGNACIONES DOCENTES,JS,3116970,NaN,NaN,No,20275425894,2026-02-28T00:00:00Z,T,...,Relevo por desempeno jerarquico sin estabilida...,BERAZATEGUI,17 A 18,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T10:31:29.483Z,1826390706449547266
30,Publicada,DESIGNACIONES DOCENTES,JS,3116201,NaN,15:10 A 17:10,No,20147633158,9999-12-30T12:26:54.217Z,T,...,NaN,BERAZATEGUI,NaN,A,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:26:55.477Z,1826390706581667843
31,Publicada,DESIGNACIONES DOCENTES,JS,3116300,16:55 A 17:55,NaN,No,20147633158,2025-04-11T00:00:00Z,T,...,Licencia no medica por ARTICULO 114(ART114NOme...,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:34:37.213Z,1826390706581667844
41,Publicada,DESIGNACIONES DOCENTES,JS,3117150,NaN,NaN,No,20275425894,9999-12-30T13:43:48.367Z,V,...,NaN,BERAZATEGUI,NaN,E,19:40 A 21:25,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T10:46:08.783Z,1826390707373342721
48,Publicada,DESIGNACIONES DOCENTES,JS,3121309,NaN,NaN,No,20275425894,9999-12-30T14:19:08.363Z,T,...,NaN,BERAZATEGUI,NaN,E,15:10 A 17:10,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T11:19:06.323Z,1826390707911262209


In [41]:
# files = os.listdir("./DATA/")
# files
# files = os.listdir("./DATA/")
# files.sort(reverse=True)
# file = files[0] #me quedo con el último
# csv_file = f'./DATA/{file}'  # Replace with the path to your CSV file
# df = pd.read_csv(csv_file)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 3 to 252
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   estado                 27 non-null     object 
 1   tipooferta             27 non-null     object 
 2   jornada                27 non-null     object 
 3   ige                    27 non-null     int64  
 4   miercoles              10 non-null     object 
 5   martes                 9 non-null      object 
 6   acargodireccion        27 non-null     object 
 7   cuilautor              27 non-null     int64  
 8   supl_hasta             27 non-null     object 
 9   turno                  27 non-null     object 
 10  idoferta               27 non-null     int64  
 11  domiciliodesempeno     27 non-null     object 
 12  cuilganador            0 non-null      float64
 13  sabado                 0 non-null      float64
 14  listadoorigenganador   0 non-null      float64
 15  id          

In [83]:
orden_distritos = {'QUILMES':2,'AVELLANEDA':1,'BERAZATEGUI':0}
ofertas_por_distrito = df.groupby(['descdistrito','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_distrito.columns = ['Distrito','Cargo','Cantidad']
ofertas_por_distrito["order"] = ofertas_por_distrito.Distrito.map(orden_distritos)
ofertas_por_distrito = ofertas_por_distrito.sort_values(by=['order','Cantidad'],ascending=False)
ofertas_por_distrito.drop(columns="order",inplace=True)
ofertas_por_distrito = tabulate(
    ofertas_por_distrito, 
    headers=ofertas_por_distrito.columns, 
    tablefmt="html",
    showindex=False,
)
ofertas_por_distrito


Distrito,Cargo,Cantidad
QUILMES,CONSTRUCCION DE CIUDADANIA (CCD),2
QUILMES,PRECEPTOR (/PR),2
BERAZATEGUI,CONSTRUCCION DE CIUDADANIA (CCD),21
BERAZATEGUI,"COMUNICACIÓN, CULTURA Y SOCIEDAD (CCS)",1
BERAZATEGUI,PRECEPTOR (/PR),1


In [84]:
ofertas_por_escuela = df.groupby(['descdistrito','escuela','domiciliodesempeno','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_escuela.columns = ['Distrito','Escuela','Dirección','Cargo','Cantidad']
ofertas_por_escuela["order"] = ofertas_por_escuela.Distrito.map(orden_distritos)
ofertas_por_escuela = ofertas_por_escuela.sort_values(by=['order','Cantidad'],ascending=False)
ofertas_por_escuela.drop(columns="order",inplace=True)
ofertas_por_escuela = tabulate(
    ofertas_por_escuela, 
    headers=ofertas_por_escuela.columns, 
    tablefmt="html",
    showindex=False,
)
ofertas_por_escuela

Distrito,Escuela,Dirección,Cargo,Cantidad
QUILMES,0085JI0929,856 E/ 883 Y 884,PRECEPTOR (/PR),1
QUILMES,0085MS0003,CORRIENTES ESQ QUINTANA,CONSTRUCCION DE CIUDADANIA (CCD),1
QUILMES,0085MS0016,CALCHAQUI Y BOMBERO SANCHEZ,CONSTRUCCION DE CIUDADANIA (CCD),1
QUILMES,0085MS0024,880 esquina 833,PRECEPTOR (/PR),1
BERAZATEGUI,0119MS0023,163 E/ 23 Y 24,CONSTRUCCION DE CIUDADANIA (CCD),3
BERAZATEGUI,0119MS0005,148,CONSTRUCCION DE CIUDADANIA (CCD),2
BERAZATEGUI,0119MS0018,27 E/ 106 Y 107,CONSTRUCCION DE CIUDADANIA (CCD),2
BERAZATEGUI,0119MS0022,129,CONSTRUCCION DE CIUDADANIA (CCD),2
BERAZATEGUI,0119MS0002,304,CONSTRUCCION DE CIUDADANIA (CCD),1
BERAZATEGUI,0119MS0005,148,PRECEPTOR (/PR),1


In [85]:
cantidad_ofertas = df["ige"].nunique()
cantidad_ofertas

27